In [18]:
# !pip install gdown

In [19]:
# !gdown https://drive.google.com/uc?id=11AENyWl6fD5rgoIIwnZy692vE_pqZQ8Z

In [20]:
# !gdown https://drive.google.com/uc?id=1ZCgMnlPTQvmU4KB5HQpLGFNZt-uEzNks

In [21]:
# !apt-get install unzip

In [22]:
# !unzip zipfile.zip -d .

In [23]:
# !rm zipfile.zip

In [24]:
# !pip install pandas razdel tqdm python-Levenshtein

In [25]:
# !pip install transformers

In [26]:
# !pip install pytorch_pretrained_bert

In [30]:
# !pip install sklearn

In [1]:
import pandas as pd
from main import QADataset
from razdel import tokenize
import torch
from torch.utils.data import Dataset
from tqdm.auto import tqdm
from preprocess_dataset import tokenize_text
from torch.utils.data import Dataset, DataLoader
import torch
from pytorch_pretrained_bert import BertModel, BertTokenizer, BertForMaskedLM, BertConfig
from get_bert_embeddings import get_embeddings, embed_data

In [2]:
data = pd.read_csv("sdsj2017_sberquad_with_spans(minus_30_examples).csv")
data.head()

,paragraph_id,question_id,paragraph,question,answer,char_answer_span,paragraph_tokens,word_answer_span
0,14754,60544,Первые упоминания о строении человеческого тел...,Где встречаются первые упоминания о строении ч...,в Древнем Египте,"(60, 76)",Первые упоминания о строении человеческого тел...,"7,10"
1,13859,1604,Телескоп имеет модульную структуру и содержит ...,Как называется корректирующая оптическая систе...,COSTAR,"(187, 193)",Телескоп имеет модульную структуру и содержит ...,"28,29"
2,8310,30350,Критики теории Вегенера поставили во главу угл...,Какая теория была отвергнута после смерти Веге...,теория дрейфа материков,"(335, 358)",Критики теории Вегенера поставили во главу угл...,"51,54"
3,8766,53270,При нагревании кусочки янтаря становятся очень...,Чему не уступают по красоте изделия из прессов...,изделиям из монолитных камней,"(852, 881)",При нагревании кусочки янтаря становятся очень...,"128,132"
4,14719,75698,Нисходящие дифтонги со слабым гласным /i/ и /u...,На какие группы классифицируют дифтонги?,оральные и назальные,"(735, 755)",Нисходящие дифтонги со слабым гласным / i / и ...,"141,144"


In [3]:
par_tokens = [i.split() for i in data.paragraph_tokens]
que_tokens = [tokenize_text(i) for i in data.question]
answer_spans = data.word_answer_span

In [4]:
word2index={"[PAD]":0, "[CLS]":1, "[SEP]":2}

for sent in par_tokens:
    for token in sent:
        if token not in word2index:
            word2index[token] = len(word2index)

for que in que_tokens:
    for token in que:
        if token not in word2index:
            word2index[token] = len(word2index)

In [5]:
len(word2index)

155047

In [6]:
tokenizer = BertTokenizer.from_pretrained("lm", do_lower_case=False)

In [7]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('Bummer!  Training on CPU ...')
else:
    print('You are good to go!  Training on GPU ...')

You are good to go!  Training on GPU ...


# Loading embeddings

In [8]:
# embeddings = embed_data(dataset.x_data)
# тут все ок

# Разбиваем датасет на трейн/тест (80%, 20%)

In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [10]:
len(train)

40267

In [11]:
train.head()

,paragraph_id,question_id,paragraph,question,answer,char_answer_span,paragraph_tokens,word_answer_span
0,11398,27507,"Идея о том, что человечество развивается по пу...",Кто первым изложил последовательную теорию про...,аббат Сен-Пьер,"(288, 302)","Идея о том , что человечество развивается по п...","48,50"
1,8918,59781,Когда на трон развивающейся страны взошёл царь...,В какой период Болгарское царство достигло апо...,период правления Симеона,"(136, 160)",Когда на трон развивающейся страны взошёл царь...,"23,26"
2,4245,14011,Углеводы — весьма обширный класс органических ...,какие главные источники углеводов в пище являю...,"хлеб, картофель, макароны, крупы, сладости","(732, 774)",Углеводы — весьма обширный класс органических ...,"109,118"
3,8258,3495,"Речно́й о́кунь[1], или обыкнове́нный окунь[2] ...",Какой вид окуней есть в водоёмах Северной Амер...,жёлтый окунь (Perca flavescens),"(524, 555)","Речно́й о́кунь [ 1 ] , или обыкнове́нный окунь...","91,97"
4,7381,32304,Подавляющее большинство правивших в Северной А...,Последователи Абдуллы ибн Ясина стали основате...,Альморавидов,"(120, 132)",Подавляющее большинство правивших в Северной А...,"15,16"


In [12]:
len(test)

10067

In [12]:
par_tokens_train = [i.split() for i in train.paragraph_tokens]
que_tokens_train = [tokenize_text(i) for i in train.question]
answer_spans_train = train.word_answer_span

train_data = QADataset(tokenizer=tokenizer,
                   paragraph_tokens=par_tokens_train,
                   question_tokens=que_tokens_train,
                   answer_spans=answer_spans_train,
                   word2index=word2index)

In [13]:
par_tokens_test = [i.split() for i in test.paragraph_tokens]
que_tokens_test = [tokenize_text(i) for i in test.question]
answer_spans_test = test.word_answer_span

test_data = QADataset(tokenizer=tokenizer,
                   paragraph_tokens=par_tokens_test,
                   question_tokens=que_tokens_test,
                   answer_spans=answer_spans_test,
                   word2index=word2index)

In [14]:
# 1. вставить эмбеддинги берта в классификатор
# 2. написать бейзлайн


# class Classifier

In [54]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertForMaskedLM

class Classifier(torch.nn.Module):
    
    def __init__(self, 
               hidden_size=3072,  
               linear_out=512,
               batch_first=True):
  
        super(Classifier, self).__init__()
            
        self.output_model_file = "lm/pytorch_model.bin"
        self.output_config_file = "lm/config.json"
        self.tokenizer = BertTokenizer.from_pretrained("lm", do_lower_case=False)
        self.config = BertConfig.from_json_file(self.output_config_file)
        self.model = BertForMaskedLM(self.config)
        self.state_dict = torch.load(self.output_model_file, map_location=device)
        self.model.load_state_dict(self.state_dict)
        
        self.linear_1 = torch.nn.Linear(hidden_size, linear_out)
        self.linear_2 = torch.nn.Linear(hidden_size, linear_out)
        
    
    def get_embeddings(self, x_instance):
        indexed_tokens = x_instance.tolist()
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_ids = [1] * len(indexed_tokens)
        segments_tensors = torch.tensor([segments_ids])
        self.model.eval()
        with torch.no_grad():
            encoded_layers, _ = self.model.bert(tokens_tensor,
                                       segments_tensors)
        token_embeddings = torch.stack(encoded_layers, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1, 0, 2)
        token_vecs_cat = []
        for token in token_embeddings:
            cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]),
                                dim=0)
            token_vecs_cat.append(cat_vec)
        token_vecs_cat = torch.stack(token_vecs_cat, dim=0)
        sentence_embedding = torch.mean(token_vecs_cat, dim=0)

        return sentence_embedding

    
    def embed_data(self, x): 
        entries = [] 
        data_iterator = tqdm(x, desc='Loading embeddings')    
        for entry in data_iterator:
            emb = self.get_embeddings(entry)
            entries.append(emb)
        return torch.stack(entries)
    
        
    def forward(self, x):

        h = self.embed_data(x)
        pred_1 = self.linear_1(h)
        pred_2 = self.linear_2(h)
        
        return pred_1, pred_2

In [56]:
test_loader = DataLoader(test_data, batch_size=32, drop_last=True)

In [57]:
for x,y in test_loader:
    pass

In [58]:
model = Classifier()

In [93]:
pred_1, pred_2 = model.forward(x)

In [29]:
train_loader = DataLoader(train_data, batch_size=32, drop_last=True)
test_loader = DataLoader(test_data, batch_size=32, drop_last=True)

epochs = 5

device = ('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = Classifier()
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=8e-6, weight_decay=0.01)

iteration_losses = []
epoch_train_losses = []
epoch_test_losses = []
accuracies = []
f_scores = []

In [83]:
from sklearn.metrics import accuracy_score, f1_score
from tqdm.auto import tqdm

best_test_loss = 1.7 # для early stopping

for n_epoch in range(epochs):
        
    train_losses = []
    test_losses = []
    test_targets = []
    test_preds = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))

    for x, y in test_loader:
        
        optimizer.zero_grad()
        pred_1, pred_2 = model.forward(x.to(device))  
        
#        y_vector = [0 for i in range(512)]
#        y_vector[int(y[0])] = 1
#        y_1 = y_vector
        
#        y_vector = [0 for i in range(512)]
#        y_vector[int(y[1])] = 1
#        y_2 = y_vector
        
        loss_1 = criterion(pred_1, y_1.long().to(device))  
        loss_2 = criterion(pred_2, y_2.long().to(device))  
        loss = loss_1 + loss_2
        loss.backward()
        optimizer.step() 
        
        iteration_losses.append(loss.item())
        train_losses.append(loss.item())
        progress_bar.set_postfix(loss=np.mean(iteration_losses[-500:]))
        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    epoch_train_losses.append(np.mean(train_losses))
  
 #    for x_test, y_test in test_loader:
 #        with torch.no_grad():
 #           pred = model.forward(x_test.to(device))
 #           test_preds.append(pred.cpu()) 
 #       test_targets.append(y_test.long())
 #       loss = criterion(pred, y_test.long().to(device))
 #       test_losses.append(loss.item()) 
        
 #       mean_test_loss = np.mean(test_losses)
 #   epoch_test_losses.append(mean_test_loss)
    
 #   print('Training: train loss = {:.3f}, test loss = {:.3f} '.format(np.mean(train_losses), mean_test_loss))
 #   print("Evaluating...")
    
 #   test_preds_cat = torch.cat(test_preds, dim=0)
 #   test_preds_argmax = torch.argmax(test_preds_cat, dim=1)
 #   test_targets = torch.cat(test_targets, dim=0)  

 #   accuracy = accuracy_score(test_targets, test_preds_argmax)
 #   f_score = f1_score(test_targets, test_preds_argmax, average='macro')
 #   accuracies.append(accuracy)
 #   f_scores.append(f_score)
 #   print('Accuracy = {:.3f}, f1-score = {:.3f}'.format(accuracy, f_score))
    
 #   if mean_test_loss < best_test_loss:
 #       best_test_loss = mean_test_loss
 #   else:
 #       print('Early stopping')
 #       break

NameError: name 'y_1' is not defined